In [ ]:
import xarray as xr
import plotdata as pd

In [ ]:
# pull data from files
try:
    ds = xr.open_dataset('nc/master')
    ds_plev = xr.open_dataset('nc/master_plev')
except FileNotFoundError:
    !python loaddata.py

In [ ]:
# get time mean for variable
tm = ds.mean(dim='time', skipna=True)

# operations on variables
w = tm['pr']

# calculate time mean anomalies
z  = w.sel(exp='LandControl') - w.sel(exp='AquaControl')

pd.contourmap_allexperiments(z)

In [ ]:
pd.quivermap_allexperiments(5, anom=True, plotname='quiver_overlay')

In [ ]:
# specify variable to plot
var = 'hfss'

# get time mean for variable
tm = ds[var].mean(dim='time', skipna=True)

# calculate time mean anomalies
z  = tm.sel(exp='AquaControl')

# plotting
g = z.plot.contourf(col='model', col_wrap=3, robust=True, aspect=2, levels=15)
for ax in g.axes.flat:
    ax.add_patch(Rectangle((0, -30), 45, 60, alpha=1, facecolor='none',
                           edgecolor='black', linewidth=1))

In [ ]:
# specify variable to plot
var = 'ts'

# calculate monthly anomalies for variable
w = ds[var].sel(exp='LandControl') - ds[var].sel(exp='AquaControl')

# get zonal mean over land and ocean
land = w.sel(lon=slice(0, 45)).mean(dim='lon', skipna=True)
ocean = xr.concat([w.sel(lon=slice(-180, 0)), w.sel(lon=slice(45, 180))], dim='lon').mean(dim='lon', skipna=True)

# plotting
land.plot(col='model', col_wrap=3, robust=True, aspect=2)

In [ ]:
# get wind data at 92500
plev = 92500

u = ds_plev['ua'].mean(dim='time', skipna=True).sel(plev=plev)
v = ds_plev['va'].mean(dim='time', skipna=True).sel(plev=plev)

# calculate anomalies
u = u.sel(exp='LandControl') - u.sel(exp='AquaControl')
v = v.sel(exp='LandControl') - v.sel(exp='AquaControl')

# plotting
plt.figure(figsize=(18, 20), dpi=80, facecolor='w', edgecolor='k')

step = 6

for i in range(12):
    u_i = u.isel(model=i)
    v_i = v.isel(model=i)
    ax = plt.subplot(5, 3, i+1)
    ax.add_patch(Rectangle((0, -30), 45, 60, alpha=1, facecolor='none',
                           edgecolor='black', linewidth=1))
    plt.quiver(u.lon[::step], u.lat[::step], u_i[::step, ::step], v_i[::step, ::step],
               units='width', pivot='mid')

plt.savefig('figs/quiver.pdf')